<h1 align="center">
  Nienadzorowana reprezentacja autoenkodery i modele generatywne
</h1>

<h4 align="center">
  12.10.2023
</h4>
<br/>



# Techniki regularyzacyjne - [dropout](https://paperswithcode.com/method/dropout)


### Importowanie niezbędnych modułow

In [ ]:
import numpy as np
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm.notebook import tqdm

Klasa ,,*AverageMeter*'' przechowuje oraz przetwarza częściowe wyniki zapisywane w poszczególnych etapach uczenia modelu. Funkcja ,,*accuracy*'' liczy dokładność sieci w procesie uczenia lub jej walidacji.

In [ ]:
class AverageMeter:
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum = self.sum + val * n
        self.count = self.count + n
        self.avg = self.sum / self.count


def accuracy(output, target, topk=[1]):
    max_k = np.max(topk)
    batch_size = target.shape[0]

    _, pred = output.topk(max_k, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    results = []
    for k in topk:
        correct_k = correct[:k].reshape(-1).float().sum(0)
        results.append(correct_k.mul_(100.0 / batch_size))
    return results

W tej części przygotowujemy zbiór danych do trenowania i walidacji modelu. Przetwarzamy obrazki ze zbioru *Cifar-10* do tensorów, które są pobierane iteracyjnie w batchach podczas trenowania sieci (zmienne: ,,*train_loader*'', ,,*test_loader*'').

In [ ]:
root = "../datasets"
download = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

trainset = datasets.CIFAR10(f"{root}/cifar10", train=True, download=download, transform=transform)
train_loader = torch.utils.data.DataLoader(
    trainset, batch_size=64, shuffle=True, num_workers=2
)

testset = datasets.CIFAR10(f"{root}/cifar10", train=False, download=download, transform=transform)
test_loader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2
)

In [ ]:
total_scores = {}

# Model bez regularyzacji dropout 

Poniżej definujemy klasy *Net*, w której tworzymy sieć neuronową bez regularyzacji *dropout*.

In [ ]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.layers = torch.nn.Sequential(
            torch.nn.Conv2d(3, 32, 3),
            torch.nn.ReLU(True),
            torch.nn.MaxPool2d(2, 2),
            torch.nn.Conv2d(32, 64, 3),
            torch.nn.ReLU(True),
            torch.nn.MaxPool2d(2, 2),
            torch.nn.Conv2d(64, 128, 3),
            torch.nn.ReLU(True),
            torch.nn.MaxPool2d(2, 2),
            torch.nn.Flatten(),
            torch.nn.Linear(512, 256),
            torch.nn.ReLU(True),
            torch.nn.Linear(256, 10),
        )

    def forward(self, x):
        return self.layers(x)

Uczenie modelu i jego walidacja.

In [ ]:
model = Net().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

scores = {
    "train": {"loss": [], "Prec@1": [], "Prec@5": []},
    "test": {"loss": [], "Prec@1": [], "Prec@5": []},
}

print("The results are described by: (loss, Prec@1, Prec@5)")
epochs = 4
for epoch in range(epochs):

    # training
    model.train()

    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    train_tqdm = tqdm(train_loader, total=len(train_loader), leave=False)
    for image, target in train_tqdm:
        image, target = image.to(device), target.to(device)

        output = model(image)
        loss = criterion(output, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        prec1, prec5 = accuracy(torch.softmax(output, dim=1), target, topk=(1, 5))
        losses.update(loss.item(), image.shape[0])
        top1.update(prec1.item(), image.shape[0])
        top5.update(prec5.item(), image.shape[0])

        train_tqdm.set_description(
            f"TRAIN [loss, P@1, P@5]: "
            f"[{losses.val:.4f} ({losses.avg:.4f}),{top1.val:2.2f} "
            f"({top1.avg:2.2f}),{top5.val:2.2f} ({top5.avg:2.2f})]"
        )

    scores["train"]["loss"].append(losses.avg)
    scores["train"]["Prec@1"].append(top1.avg)
    scores["train"]["Prec@5"].append(top5.avg)

    # validating
    model.eval()

    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    with torch.no_grad():
        eval_tqdm = tqdm(test_loader, total=len(test_loader), leave=False)
        for image, target in eval_tqdm:
            image, target = image.to(device), target.to(device)

            output = model(image)
            loss = criterion(output, target)

            prec1, prec5 = accuracy(torch.softmax(output, dim=1), target, topk=(1, 5))
            losses.update(loss.item(), image.shape[0])
            top1.update(prec1.item(), image.shape[0])
            top5.update(prec5.item(), image.shape[0])

            eval_tqdm.set_description(
                f"TEST [loss, P@1, P@5]: "
                f"[{losses.val:.4f} ({losses.avg:.4f}),{top1.val:2.2f} "
                f"({top1.avg:2.2f}),{top5.val:2.2f} ({top5.avg:2.2f})]"
            )

    scores["test"]["loss"].append(losses.avg)
    scores["test"]["Prec@1"].append(top1.avg)
    scores["test"]["Prec@5"].append(top5.avg)

    print(
        f"Epoch: [{epoch + 1}/{epochs}]\t"
        f"train: ({scores['train']['loss'][-1]:.4f},{scores['train']['Prec@1'][-1]:2.2f},"
        f"{scores['train']['Prec@5'][-1]:2.2f})\t"
        f"test: ({scores['test']['loss'][-1]:.4f},{scores['test']['Prec@1'][-1]:2.2f},"
        f"{scores['test']['Prec@5'][-1]:2.2f})"
    )

In [ ]:
total_scores['without_dropout'] = scores

# Model z regularyzacją dropout

Poniżej definujemy klasy *Net*, w której tworzymy sieć neuronową z regularyzacją *dropout*.

In [ ]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.layers = torch.nn.Sequential(
            torch.nn.Conv2d(3, 32, 3),
            torch.nn.ReLU(True),
            torch.nn.MaxPool2d(2, 2),
            torch.nn.Conv2d(32, 64, 3),
            torch.nn.ReLU(True),
            torch.nn.MaxPool2d(2, 2),
            torch.nn.Conv2d(64, 128, 3),
            torch.nn.ReLU(True),
            torch.nn.MaxPool2d(2, 2),
            torch.nn.Flatten(),
            torch.nn.Dropout(0.25),
            torch.nn.Linear(512, 256),
            torch.nn.ReLU(True),
            torch.nn.Dropout(0.25),
            torch.nn.Linear(256, 10),
        )

    def forward(self, x):
        return self.layers(x)

Uczenie modelu i jego walidacja.

In [ ]:
model = Net().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

scores = {
    "train": {"loss": [], "Prec@1": [], "Prec@5": []},
    "test": {"loss": [], "Prec@1": [], "Prec@5": []},
}

print("The results are described by: (loss, Prec@1, Prec@5)")
epochs = 4
for epoch in range(epochs):

    # training
    model.train()

    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    train_tqdm = tqdm(train_loader, total=len(train_loader), leave=False)
    for image, target in train_tqdm:
        image, target = image.to(device), target.to(device)

        output = model(image)
        loss = criterion(output, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        prec1, prec5 = accuracy(torch.softmax(output, dim=1), target, topk=(1, 5))
        losses.update(loss.item(), image.shape[0])
        top1.update(prec1.item(), image.shape[0])
        top5.update(prec5.item(), image.shape[0])

        train_tqdm.set_description(
            f"TRAIN [loss, P@1, P@5]: "
            f"[{losses.val:.4f} ({losses.avg:.4f}),{top1.val:2.2f} "
            f"({top1.avg:2.2f}),{top5.val:2.2f} ({top5.avg:2.2f})]"
        )

    scores["train"]["loss"].append(losses.avg)
    scores["train"]["Prec@1"].append(top1.avg)
    scores["train"]["Prec@5"].append(top5.avg)

    # validating
    model.eval()

    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    with torch.no_grad():
        eval_tqdm = tqdm(test_loader, total=len(test_loader), leave=False)
        for image, target in eval_tqdm:
            image, target = image.to(device), target.to(device)

            output = model(image)
            loss = criterion(output, target)

            prec1, prec5 = accuracy(torch.softmax(output, dim=1), target, topk=(1, 5))
            losses.update(loss.item(), image.shape[0])
            top1.update(prec1.item(), image.shape[0])
            top5.update(prec5.item(), image.shape[0])

            eval_tqdm.set_description(
                f"TEST [loss, P@1, P@5]: "
                f"[{losses.val:.4f} ({losses.avg:.4f}),{top1.val:2.2f} "
                f"({top1.avg:2.2f}),{top5.val:2.2f} ({top5.avg:2.2f})]"
            )

    scores["test"]["loss"].append(losses.avg)
    scores["test"]["Prec@1"].append(top1.avg)
    scores["test"]["Prec@5"].append(top5.avg)

    print(
        f"Epoch: [{epoch + 1}/{epochs}]\t"
        f"train: ({scores['train']['loss'][-1]:.4f},{scores['train']['Prec@1'][-1]:2.2f},"
        f"{scores['train']['Prec@5'][-1]:2.2f})\t"
        f"test: ({scores['test']['loss'][-1]:.4f},{scores['test']['Prec@1'][-1]:2.2f},"
        f"{scores['test']['Prec@5'][-1]:2.2f})"
    )

In [ ]:
total_scores['with_dropout'] = scores

Poniżej przedstawiamy wyniki powyższych dwóch modeli w postaci wykresów. Na pierwszym pokazujemy dokładność modeli w poszczególnych etapach ich uczenia, zaś na drugim wykresie przedstawiamy jak zmienia się funkcja kosztu w czasie ich uczenia.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(font_scale=2.5)
sns.set_style("whitegrid")

In [ ]:
# Prec@1
fig = plt.figure(figsize=(22, 8))
ax = fig.add_subplot(111)
ax.plot(
    total_scores["without_dropout"]["train"]["Prec@1"],
    "g--",
    linewidth=4,
    markersize=12,
    label="Model bez dropout uczony na zbiorze treningowym",
)
ax.plot(
    total_scores["without_dropout"]["test"]["Prec@1"],
    "g-",
    linewidth=4,
    markersize=12,
    label="Model bez dropout uczony na zbiorze testowym",
)
ax.plot(
    total_scores["with_dropout"]["train"]["Prec@1"],
    "b--",
    linewidth=4,
    markersize=12,
    label="Model z dropout uczony na zbiorze treningowym",
)
ax.plot(
    total_scores["with_dropout"]["test"]["Prec@1"],
    "b-",
    linewidth=4,
    markersize=12,
    label="Model z dropout uczony na zbiorze testowym",
)
ax.tick_params(
    axis="both",
    which="both",
    direction="out",
    length=6,
    width=2,
    colors="k",
    grid_alpha=0.5,
)
ax.grid(which="both")
ax.grid(which="major", color="#CCCCCC", linestyle="--", alpha=0.8)
ax.grid(which="minor", color="#CCCCCC", linestyle=":", alpha=0.8)

ax.legend(loc=0)
ax.set_ylabel("ACC (%)")
ax.set_xlabel("Epoka")
plt.tight_layout(pad=0.5)

plt.show()
plt.close()


# loss
fig = plt.figure(figsize=(22, 8))
ax = fig.add_subplot(111)
ax.plot(
    total_scores["without_dropout"]["train"]["loss"],
    "g--",
    linewidth=4,
    markersize=12,
    label="Model bez dropout uczony na zbiorze treningowym",
)
ax.plot(
    total_scores["without_dropout"]["test"]["loss"],
    "g-",
    linewidth=4,
    markersize=12,
    label="Model bez dropout uczony na zbiorze testowym",
)
ax.plot(
    total_scores["with_dropout"]["train"]["loss"],
    "b--",
    linewidth=4,
    markersize=12,
    label="Model z dropout uczony na zbiorze treningowym",
)
ax.plot(
    total_scores["with_dropout"]["test"]["loss"],
    "b-",
    linewidth=4,
    markersize=12,
    label="Model z dropout uczony na zbiorze testowym",
)
ax.tick_params(
    axis="both",
    which="both",
    direction="out",
    length=6,
    width=2,
    colors="k",
    grid_alpha=0.5,
)
ax.grid(which="both")
ax.grid(which="major", color="#CCCCCC", linestyle="--", alpha=0.8)
ax.grid(which="minor", color="#CCCCCC", linestyle=":", alpha=0.8)

ax.legend(loc=0)
ax.set_ylabel("Loss")
ax.set_xlabel("Epoka")
plt.tight_layout(pad=0.5)

plt.show()
plt.close()